<a href="https://colab.research.google.com/github/CodeSagePath/CodeSagePath/blob/master/Deep_CNN_Image_Classifier_For_2D_Room_Layouts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Setup**

In [1]:
import tensorflow as tf
import os

# import zipfile
# zip_path = '/content/ezyzip.zip'
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall('/content')

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU') #shows all gpus available
#this is telling tensorflow to limit the memory consumption
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

# **Remove Bad Images**

In [3]:
import cv2
import imghdr

In [7]:
!rm '/content/data/.DS_Store'
data_dir = '/content/data'
image_exts = ['jpeg', 'jpg', 'bmp', 'png']

rm: cannot remove '/content/data/.DS_Store': No such file or directory


In [6]:
for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e:
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/data'

Load Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#building data pipeline
data = tf.keras.utils.image_dataset_from_directory('/content/data')

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
batch[0].shape

In [ ]:
# Class 1 = 2D Office Spaces
# Class 0 = 2D Living Spaces
batch[1]

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for index, img in enumerate(batch[0][:4]):
    ax[index].imshow(img.astype(int))
    ax[index].title.set_text(batch[1][index])

# **Preprocess Data**

In [ ]:
#scaling the data
data = data.map(lambda x,y: ((x/255), y)) #scales all the pixel values to between 0 and 1, map function allows us to apply the scaling transformation as the data is accessed.

In [ ]:
data.as_numpy_iterator().next()

In [ ]:
#splitting data into training, validation, and test partition
train_size = int(len(data)*.7) #used to train our model
val_size = int(len(data)*.2) + 1 #used to evaluate our model during training and fine tune
test_size = int(len(data)*.1) + 1 #used for post training evaluation
print(len(data) == (train_size + val_size + test_size))

In [ ]:
#data is already shuffled
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)
print(len(data) == len(train) + len(val) + len(test))

# **Building The Deep Learning Model**

In [ ]:
from tensorflow.keras.layers import RandomFlip, RandomRotation, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers.experimental.preprocessing import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler

In [ ]:
# Data Augmentation
data_augmentation = Sequential([
    RandomFlip("horizontal_and_vertical"),
    RandomRotation(0.2),
    RandomZoom(0.2),
])

model = Sequential([
    data_augmentation,  # Data augmentation layers
    Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    BatchNormalization(),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.build(input_shape=(None, 256, 256, 3))  # Explicitly building the model
model.summary()

# **Training**

In [ ]:
logdir = 'logs'
tensorboard_callback = TensorBoard(log_dir=logdir)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=0.00001,
    verbose=1
)

hist = model.fit(
    train,
    epochs=30,
    validation_data=val,
    callbacks=[tensorboard_callback, reduce_lr]
)

# **Plot Performance**

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# **Evaluate**

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
precs = Precision()
re = Recall()
acc = BinaryAccuracy()

for batch in test.as_numpy_iterator():
    X, y = batch
    yhat = model.predict(X)
    precs.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f'Precision: {precs.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

# **Test**

In [ ]:
imgliving1 = cv2.imread('/content/data/test/test-livingspace1.jpg')
imgliving2 = cv2.imread('/content/data/test/test-livingspace2.jpg')
imgoffice1 = cv2.imread('/content/data/test/test-office1.jpg')
imgoffice2 = cv2.imread('/content/data/test/test-office2.jpg')

In [ ]:
plt.imshow(imgliving1)
plt.show()

In [ ]:
resize_imgliving1 = tf.image.resize(imgliving1, (256, 256))
resize_imgliving2 = tf.image.resize(imgliving2, (256, 256))
resize_imgoffice1 = tf.image.resize(imgoffice1, (256, 256))
resize_imgoffice2 = tf.image.resize(imgoffice2, (256, 256))

plt.imshow(resize_imgliving1.numpy().astype(int))
plt.show()

In [ ]:
np.expand_dims(resize_imgliving1, 0)
np.expand_dims(resize_imgliving2, 0)
np.expand_dims(resize_imgoffice1, 0)
np.expand_dims(resize_imgoffice2, 0)

In [ ]:
yhat_living1 = model.predict(np.expand_dims(resize_imgliving1/255, 0))
yhat_living2 = model.predict(np.expand_dims(resize_imgliving2/255, 0))
yhat_office1 = model.predict(np.expand_dims(resize_imgoffice1/255, 0))
yhat_office2 = model.predict(np.expand_dims(resize_imgoffice2/255, 0))
print("Living Space = 0 \nOffice Space = 1")
print(f'Living Space Test #1: {yhat_living1[0][0]} \nLiving Space Test #2: {yhat_living2[0][0]} \nOffice Space Test #1: {yhat_office1[0][0]} \nOffice Space Test #2: {yhat_office2[0][0]}')

future considerations: make sure that when preprocessing data, remove all color, otherwise the model may form biases.